In [1]:
import tensorflow as tf
import numpy as np
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
np.random.seed(100)

4 Physical GPUs, 1 Logical GPU


In [2]:
import os
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input,Activation,GRU
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
from joblib import Parallel,delayed
from scipy.stats import mode

In [3]:
X,y = pickle.load(open('./data/10/walking_all_data1.p','rb'))

In [4]:
def get_training_testing_sitting(window_size=10,
                         base_window_size=10,n_user=300):
    n_train = window_size*60//base_window_size
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    count = 0
    i = 0
    while i<len(X):
        if X[i].shape[0]<n_train+30*6:
            i+=1
            continue
        X_train.append(X[i][:n_train])
        X_test.append(X[i][n_train:])
        y_train.extend([y[i]]*X_train[-1].shape[0])
        y_test.extend([y[i]]*X_test[-1].shape[0])
        count+=1
        i+=1
        if count==n_user:
            break
    return np.concatenate(X_train),np.concatenate(X_test),y_train,y_test
def do_averaging(X1,w=30):
    def moving_average(x, w):
        return pd.DataFrame({'x':x}).x.rolling(window=w).mean().fillna(method='bfill').values.reshape(-1)
#         return np.convolve(x, np.ones(w), 'same') / w
    X2 = X1.copy()
    for i in range(X2.shape[0]):
        X2[i] = np.apply_along_axis(lambda m: moving_average(m,w), axis=0, arr=X2[i])
    return X2

def get_model(input_shape=(500,3),n_classes=1):
    model =  Sequential()
    model.add(Conv1D(128,2,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
    model.add(Activation('tanh'))
#     model.add(BatchNormalization())
    model.add(Dropout(.5))
    model.add(GRU(128,return_sequences=True,activation='tanh'))
    model.add(Flatten())
    model.add(Dense(n_classes))
    model.add(Dense(n_classes,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam')
    return model


def get_test_scores(model,x_test,y_test,test_size=None,n_iter=1000,base_window_size=20):
    def get_score(user,test_size,n_iter,base_window_size):
        df = test_df[test_df.original==user]
        score1 = []
        score2 = []
        test_lengths = []
        for t in test_size:
            n  = np.int64(t*60/base_window_size)
            if n>df.shape[0]:
                continue
            y_true = [df['original'].values[0]]*n_iter
            y_pred_majority = []
    #         y_pred_maxmean = []
            for i in range(n_iter):
                temp_df = df.sample(n,replace=False)
                y_pred_majority.append(mode(temp_df['prediction'].values)[0][0])
    #             temp_df = np.mean(np.concatenate(list(temp_df['probability'])),axis=0)
    #             y_pred_maxmean.append(np.argmax(temp_df))


            majority_score = accuracy_score(y_true,y_pred_majority)
    #         maxmean_score = accuracy_score(y_true,y_pred_maxmean)
            score1.append(majority_score)
    #         score2.append(maxmean_score)
            test_lengths.append(t)
        
        df = pd.DataFrame({'majority_score':score1,
    #                         'maxmean_score':score2,
                            'test_lengths':test_lengths})
#         print(df.loc[0])
        return df
    
    y_pred = model.predict(x_test).argmax(axis=1)
#     y_pred1 = y_pred.argmax(axis=1)
    test_df = pd.DataFrame({'prediction':y_pred,'original':y_test})
#                             'probability':list(y_pred)})
#     test_df['probability'] = test_df['probability'].apply(lambda a:a.reshape(1,-1)) 
    if test_size is None:
        test_size = np.arange(1,120,1)
    del x_test
    
    result  = Parallel(verbose=2,n_jobs=30)(delayed(get_score)(user,test_size,n_iter,base_window_size) for user in np.unique(test_df.original.values))
    test_score = pd.concat(result)
    acc_ = accuracy_score(test_df['original'],test_df['prediction'])
    
    test_score = test_score.append(pd.DataFrame({'majority_score':[acc_],
#                                                     'maxmean_score':[acc_],
                                                    'test_lengths':[0]}))
    test_score = test_score.groupby('test_lengths',as_index=False).mean()
    return test_score



def get_trained_model(X_train,y_train,n_timesteps,n_channels,window_size,filepath):
    n_classes = len(np.unique(y_train))
    model =  get_model(input_shape=(n_timesteps,n_channels),n_classes=n_classes)
#     model.summary()
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=False)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
    callbacks_list = [es,checkpoint]
#     train_x,test_x,train_y,test_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    train_x,val_x,train_y,val_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=200, batch_size=300,verbose=0,
          callbacks=callbacks_list,shuffle=True)
    model.load_weights(filepath)
#     y_pred = model.predict(test_x).argmax(axis=1)

#     from sklearn.metrics import classification_report

#     print(accuracy_score(test_y,y_pred),window_size)
    return model


def get_training_testing(window_size=10,
                         base_window_size=10):
    n_train = window_size*60//base_window_size
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    def split_data(df):
        filepath = directory+df['user'].values[0]
        df  = pickle.load(open(filepath,'rb'))
        if df.shape[0]<n_train+30*6:
            print(filepath[-2:],end=',')
            return 
        df['time'] = df['window'].apply(lambda a:a[0])
        df['data'] = df['data'].apply(lambda a:np.array([np.array(b) for b in a]))
        df['data'] = df['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
        df['data'] = df['data'].apply(lambda a:a[:,:,1:])
        df = df.sort_values('time').reset_index(drop=True)
        user_id = df.user.values[0]
        X_train.append(np.concatenate(list(df[:n_train]['data'].values)))
        X_test.append(np.concatenate(list(df[n_train:]['data'].values)))
        y_train.extend([user_id]*X_train[-1].shape[0])
        y_test.extend([user_id]*X_test[-1].shape[0])
        print(filepath[-2:],end=',')
        return 
    data.groupby('user',as_index=False).apply(split_data)
    return np.concatenate(X_train),np.concatenate(X_test),y_train,y_test



# directory = './data/walking_10/right_wrist/'
# filepath1 = './data/all_walking_data_10.p'
# if not os.path.isfile(filepath1):
#     print(filepath1)
#     data_col = []
#     for f in os.listdir(directory):
#         filepath = directory+f
#         data  = pickle.load(open(filepath,'rb'))
#         data_col.append(df)
#     data = pd.concat(data_col)
#     unique_users = data.user.unique()
#     unique_users_dict = {u:i for i,u in enumerate(unique_users)}
#     data['user_id'] = data['user'].apply(lambda a:unique_users_dict[a])
#     data['time'] = data['window'].apply(lambda a:a[0])
#     data['data'] = data['data'].apply(lambda a:np.array([np.array(b) for b in a]))
#     data['data'] = data['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
#     data['data'] = data['data'].apply(lambda a:a[:,:,1:])
#     pickle.dump(data,open(filepath1,'wb'))
# data = pd.DataFrame({'user':list(os.listdir(directory))})


def get_test_score(X_train,X_test,y_train,y_test,window_size,base_window_size,n_user,model_save_path,n_timesteps,n_channels,test_lengths,smoothing=10):
    model_save_path1 = model_save_path+ str(window_size)+'/'
    if not os.path.isdir(model_save_path1):
        os.makedirs(model_save_path1)        
    pickle.dump(unique_users_dict,open(model_save_path1+str(n_user)+'user_dict.p','wb'))
    filepath = model_save_path1+str(smoothing)+'samples_ma_walking.hdf5'
    model = get_trained_model(np.nan_to_num(X_train),y_train,n_timesteps,n_channels,window_size,filepath=filepath)
    test_score = get_test_scores(model,np.nan_to_num(X_test),y_test,test_size=test_lengths,n_iter=1000,base_window_size=base_window_size)
    test_score['train_user'] = n_user
    test_score['train_lengths'] = window_size
    test_score['smoothing_factor'] = smoothing    
    del X_train
    del X_test
    del y_train
    del y_test
    return test_score

base_window_size = 10
Fs = 25
n_timesteps = base_window_size*Fs
n_channels = 3
window_size = 120
test_lengths = np.arange(1,121,1)
n_user = 347
model_save_path = './models/mymodels/'
result_save_path = './results/myresults/'
if not os.path.isdir(result_save_path):
    os.makedirs(result_save_path)
if not os.path.isdir(model_save_path):
    os.makedirs(model_save_path)
X_train1,X_test1,y_train1,y_test1 = get_training_testing_sitting(window_size=window_size,base_window_size=base_window_size,n_user=n_user)
for smoothing in np.arange(0,255,2):
    if smoothing==0:
        X_train = X_train1.copy()
        X_test = X_test1.copy()
    else:    
        X_train = do_averaging(X_train1,w=smoothing)
        X_test = do_averaging(X_test1,w=smoothing)
    y_train = y_train1.copy()
    y_test = y_test1.copy()
    print(X_train.shape,X_test.shape)
    unique_users = np.unique(y_train+y_test)
    unique_users_dict = {u:i for i,u in enumerate(unique_users)}
    y_train = np.array([unique_users_dict[a] for a in y_train])
    y_test = np.array([unique_users_dict[a] for a in y_test])
    test_score = get_test_score(X_train,X_test,y_train,y_test,window_size,base_window_size,n_user,model_save_path,n_timesteps,n_channels,test_lengths,smoothing=smoothing)
#     results.append(test_score)
    pickle.dump(test_score,open(result_save_path+str(smoothing)+'.p','wb'))
    print(n_user,window_size,end=',')
    print(smoothing,'done','-'*20)

(173520, 250, 3) (433466, 250, 3)
Epoch 00058: early stopping


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  4.0min
[Parallel(n_jobs=30)]: Done 241 out of 241 | elapsed:  8.8min finished


347 120,0 done --------------------
(173520, 250, 3) (433466, 250, 3)
Epoch 00060: early stopping


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  4.0min
[Parallel(n_jobs=30)]: Done 241 out of 241 | elapsed:  8.7min finished


347 120,2 done --------------------
(173520, 250, 3) (433466, 250, 3)
Epoch 00060: early stopping


...

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  3.9min
[Parallel(n_jobs=30)]: Done 241 out of 241 | elapsed:  8.5min finished


347 120,252 done --------------------
(173520, 250, 3) (433466, 250, 3)
Epoch 00114: early stopping


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  3.9min


347 120,254 done --------------------


[Parallel(n_jobs=30)]: Done 241 out of 241 | elapsed:  8.6min finished
